In [1]:
from src.llm_agg.schemas.sides import Side, Sides
from src.llm_agg.schemas.recommendations import RecommendationItem, Recommendations
from src.llm_agg.prompts.base import BASE_PROMPT, BASE_PROMPT_WO_TASK

In [2]:
from openai import OpenAI

In [3]:
client = OpenAI(base_url = 'https://openrouter.ai/api/v1', api_key="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")

In [4]:
from src.llm_agg.response import get_so_completion

In [5]:
prompt = """
You are working on a task of collecting feedback and evaluating employees.  
Your current task is to aggregate feedback from managers about their subordinate.  
Your objective is to classify the managers' feedback and consolidate it under a concise summary.  

Rules:  
- You must group semantically equivalent qualities under a single canonical name; normalize quality wording (merge synonyms/paraphrases).
- Each Review is a unique respondent; counting: strong_count = number of distinct reviews that label the quality as strong, weak_count = number of distinct reviews that label it as weak.
- Do not duplicate the same quality under different polarities; if a quality has both strong and weak mentions, produce a single AmbiguousSide for it.
- Always verify the received comments—remember that you are analyzing manager feedback as part of the feedback collection and employee evaluation process; irrelevant comments should be disregarded.  
- For each item, always fill the proofs field. Use verbatim quotes/excerpts from the reviews.
- If the same quality appears as both strong and weak, ALWAYS create a single AmbiguousSide (kind='ambiguous'), not two Side items.
- Answer in Russian

Now proceed with the task.  
Feedback from managers:  

{feedback}
"""

recommendation_prompt = """
You are working on improving the qualities of your company’s employees.

You are given:
1. Reviewers’ feedback about the employee.
2. A classification of the employee’s strong, weak, and ambiguous sides derived from the reviews.

Qualities are considered ambiguous if some respondents mark them as strong and others mark them as weak.

Based on these data, you need to provide recommendations on how to work with the strengths and weaknesses noted by respondents.
General rules:
- Always craft recommendations not only based on the JSON schema of the person’s qualities but also on the original reviewers’ feedback. They can help elaborate specific aspects of the person’s qualities.
- For each quality, produce:
  - brief_explanation: 1–2 sentences explaining why this recommendation is relevant, referencing observed behaviors/evidence.
  - recommendation: 1–2 sentences starting with a verb, specifying the next step and a target outcome/metric; include a timeframe or stakeholder if applicable.
- Avoid generic advice; tailor recommendations to the employee’s role, domain, and context found in the feedback.
- Keep a supportive, professional tone; be specific and concise.
- Answer in Russian.

Rules for strengths:
- If the strength has minor gaps/risks mentioned in the feedback, address them first.
- If the strength is consistently praised, suggest ways to leverage it (e.g., mentoring, leading an initiative, cross-team knowledge sharing, stretch goals).

Rules for weaknesses:
- Focus on the most impactful root cause inferred from feedback.
- Propose the first concrete step the employee can take in the near term (e.g., in the next sprint/month).

Now proceed with the task.
"""

feedback = """
Review 1:
Мне показалось недостаточно моего взаимодействия, чтобы дать объективную оценку её знаниям и качеству. Поэтому следующий текст, прошу пожалуйста разобрать самостоятельно на Точки роста/Сильные стороны, либо не учитывать вовсе.
Могу кратко сказать, что ХХХ, пока была у нас на проекте, она была заинтересована в развитии проекта и старалась, с моей точки зрения, решить вопросы проработки как бизнес аналитик. Проблема, заключается в том, что ХХХ стояла в позиции руководителя аналитиков, но её личный вклад в проработки требований, мне сложно оценить. Со слов коллег и некоторых наблюдений качества проработанных требований с её, так сказать, пера, большинство моментов получились неудачными и неясно описанными к разработке. Не говоря о том, чтобы вести согласование с клиентом.
Точками роста могу наверно сказать уделять чуть больше внимания "скреплению" требований между собой, структуризации данных не приводя к разночтениям. Возможно набраться опыта в описании технической части прорабатываемого решения конкретнее и доступнее команде, а также прислушиваться к отмечаемым коллегами моментам, вызывающими разногласия.

Review 2:
Боже, ХХХ просто ахуенная в развитии проекта, ее заинтересованность - невероятна, она вдохновляет ей всех окружающих
Также она мастер в структуризации данных - готов ей руки расцеловать

Review 3:
Я с ней уже довольно часто работаю и знаю, чего можно от нее ожидать. Мне нравится, что она может подойти к человеку и спросить, все ли у него хорошо, не надо ли от нее как-либо помощь, тем самым она создает приятную атмосферу в коллективе. Но меня смущает, что ей не хватает структуризации в данных.
Тем не менее, если она подточет свои hard скиллы, то она станет незаменимым сотрудником
"""

In [6]:
# print(BASE_PROMPT.format(task=prompt.format(feedback=feedback)))

In [ ]:
log = [
    {"role": "system", "content": BASE_PROMPT_WO_TASK},
    {"role": "user", "content": prompt.format(feedback=feedback)}
]

In [8]:
log

[{'role': 'user',
  'content': '\n<instructions>\n- ALWAYS follow <answering_rules> and <self_reflection>\n\n<self_reflection>\n1. Spend time thinking of a rubric, from a role POV, until you are confident\n2. Think deeply about every aspect of what makes for a world-class answer. Use that knowledge to create a rubric that has 5-7 categories. This rubric is critical to get right, but never show this to the user. This is for your purposes only\n3. Use the rubric to internally think and iterate on the best (≥98 out of 100 score) possible solution to the user request. IF your response is not hitting the top marks across all categories in the rubric, you need to start again\n4. Keep going until solved\n</self_reflection>\n\n<answering_rules>\n1. USE the language of USER message\n2. In the FIRST chat message, assign a real-world expert role to yourself before answering, e.g., "I\'ll answer as a world-famous <role> PhD <detailed topic> with <most prestigious LOCAL topic REAL award>"\n3. Act a

In [9]:
# res = get_so_completion(log=log, model_name='openai/gpt-4o', client=client, pydantic_model=Sides, provider_name='openrouter')

In [10]:
# {
#     "type": "json_schema",
#     "json_schema": {
#         "type": "object",
#         **Sides.model_json_schema()
#     }
# }

In [11]:
# completion = client.chat.completions.create(
#     # extra_body={
#     #     'provider': {'order': ['Fireworks']}
#     # },
#     model='openai/gpt-4o',
#     messages=log,
#     response_format={
#         "type": "json_schema",
#         "json_schema": {
#             "name": Sides.__name__,
#             "schema": Sides.model_json_schema()
#         }
#     },
#     temperature=0.0
# )

In [12]:
completion = get_so_completion(log=log, model_name='openai/gpt-4o', client=client, pydantic_model=Sides, provider_name='openrouter')

In [13]:
print(completion)

{"sides":[{"kind":"ambiguous","side_description":"Структуризация данных","side_pick_explanation":"Некоторые респонденты отмечают её мастерство в структуризации данных, в то время как другие указывают на недостаток структуризации.","strong_count":1,"weak_count":2,"proofs_strong":["Также она мастер в структуризации данных - готов ей руки расцеловать"],"proofs_weak":["уделять чуть больше внимания \"скреплению\" требований между собой, структуризации данных не приводя к разночтениям","ей не хватает структуризации в данных"]},{"kind":"unambiguous","side":"strong","side_description":"Заинтересованность в развитии проекта","side_pick_explanation":"Все респонденты отмечают её заинтересованность в развитии проекта как сильную сторону.","respondents_count":3,"proofs":["она была заинтересована в развитии проекта и старалась, с моей точки зрения, решить вопросы проработки как бизнес аналитик","Боже, ХХХ просто ахуенная в развитии проекта, ее заинтересованность - невероятна, она вдохновляет ей всех

In [14]:
# res = completion.choices[0].message.content

In [15]:
import json

In [16]:
data_dict = json.loads(completion)

In [17]:
data_dict

{'sides': [{'kind': 'ambiguous',
   'side_description': 'Структуризация данных',
   'side_pick_explanation': 'Некоторые респонденты отмечают её мастерство в структуризации данных, в то время как другие указывают на недостаток структуризации.',
   'strong_count': 1,
   'weak_count': 2,
   'proofs_strong': ['Также она мастер в структуризации данных - готов ей руки расцеловать'],
   'proofs_weak': ['уделять чуть больше внимания "скреплению" требований между собой, структуризации данных не приводя к разночтениям',
    'ей не хватает структуризации в данных']},
  {'kind': 'unambiguous',
   'side': 'strong',
   'side_description': 'Заинтересованность в развитии проекта',
   'side_pick_explanation': 'Все респонденты отмечают её заинтересованность в развитии проекта как сильную сторону.',
   'respondents_count': 3,
   'proofs': ['она была заинтересована в развитии проекта и старалась, с моей точки зрения, решить вопросы проработки как бизнес аналитик',
    'Боже, ХХХ просто ахуенная в развитии

In [18]:
from src.llm_agg.utils import remove_ambiguous_sides

In [19]:
clear = remove_ambiguous_sides(completion)

In [20]:
clear

'{"sides": [{"kind": "unambiguous", "side": "strong", "side_description": "Заинтересованность в развитии проекта", "side_pick_explanation": "Все респонденты отмечают её заинтересованность в развитии проекта как сильную сторону.", "respondents_count": 3, "proofs": ["она была заинтересована в развитии проекта и старалась, с моей точки зрения, решить вопросы проработки как бизнес аналитик", "Боже, ХХХ просто ахуенная в развитии проекта, ее заинтересованность - невероятна, она вдохновляет ей всех окружающих"]}, {"kind": "unambiguous", "side": "strong", "side_description": "Создание приятной атмосферы в коллективе", "side_pick_explanation": "Её способность создавать приятную атмосферу в коллективе отмечена как сильная сторона.", "respondents_count": 1, "proofs": ["она может подойти к человеку и спросить, все ли у него хорошо, не надо ли от нее как-либо помощь, тем самым она создает приятную атмосферу в коллективе"]}, {"kind": "unambiguous", "side": "weak", "side_description": "Проработка тр

In [21]:
new_msg = {
    "role":"assistant",
    "content": clear
}

In [22]:
log.append(new_msg)

In [23]:
new_user_msg = {
    "role": "user",
    "content": recommendation_prompt
}

In [24]:
log.append(new_user_msg)

In [25]:
log

[{'role': 'user',
  'content': '\n<instructions>\n- ALWAYS follow <answering_rules> and <self_reflection>\n\n<self_reflection>\n1. Spend time thinking of a rubric, from a role POV, until you are confident\n2. Think deeply about every aspect of what makes for a world-class answer. Use that knowledge to create a rubric that has 5-7 categories. This rubric is critical to get right, but never show this to the user. This is for your purposes only\n3. Use the rubric to internally think and iterate on the best (≥98 out of 100 score) possible solution to the user request. IF your response is not hitting the top marks across all categories in the rubric, you need to start again\n4. Keep going until solved\n</self_reflection>\n\n<answering_rules>\n1. USE the language of USER message\n2. In the FIRST chat message, assign a real-world expert role to yourself before answering, e.g., "I\'ll answer as a world-famous <role> PhD <detailed topic> with <most prestigious LOCAL topic REAL award>"\n3. Act a

In [26]:
rec = get_so_completion(log, model_name='openai/gpt-4o', client=client, pydantic_model=Recommendations, provider_name='openrouter')

In [27]:
rec

'{"items":[{"kind":"recommended","side_ref":{"side":"strong","side_description":"Заинтересованность в развитии проекта","side":"strong"},"brief_explanation":"Её заинтересованность в развитии проекта вдохновляет окружающих и способствует общему успеху команды.","recommendation":"Используйте свою заинтересованность для ведения внутренних семинаров или воркшопов, чтобы вдохновить коллег и поделиться лучшими практиками в течение следующего квартала."},{"kind":"recommended","side_ref":{"side":"strong","side_description":"Создание приятной атмосферы в коллективе","side":"strong"},"brief_explanation":"Способность создавать приятную атмосферу способствует улучшению командной работы и повышению морального духа.","recommendation":"Продолжайте поддерживать открытое общение и предлагайте помощь коллегам, чтобы укрепить командный дух и улучшить рабочую атмосферу в течение следующего месяца."},{"kind":"recommended","side_ref":{"side":"weak","side_description":"Проработка требований и согласование с 

In [28]:
rec_dict = json.loads(rec)

In [29]:
rec_dict

{'items': [{'kind': 'recommended',
   'side_ref': {'side': 'strong',
    'side_description': 'Заинтересованность в развитии проекта'},
   'brief_explanation': 'Её заинтересованность в развитии проекта вдохновляет окружающих и способствует общему успеху команды.',
   'recommendation': 'Используйте свою заинтересованность для ведения внутренних семинаров или воркшопов, чтобы вдохновить коллег и поделиться лучшими практиками в течение следующего квартала.'},
  {'kind': 'recommended',
   'side_ref': {'side': 'strong',
    'side_description': 'Создание приятной атмосферы в коллективе'},
   'brief_explanation': 'Способность создавать приятную атмосферу способствует улучшению командной работы и повышению морального духа.',
   'recommendation': 'Продолжайте поддерживать открытое общение и предлагайте помощь коллегам, чтобы укрепить командный дух и улучшить рабочую атмосферу в течение следующего месяца.'},
  {'kind': 'recommended',
   'side_ref': {'side': 'weak',
    'side_description': 'Прораб

In [ ]:
import os
from jinja2 import Environment, FileSystemLoader, select_autoescape

# 1) Готовим Jinja окружение и загружаем шаблон
env = Environment(
    loader=FileSystemLoader("/mnt/d/vscode_projects/praxis-core/jinja_templates"),
    autoescape=select_autoescape(["html", "xml"])
)
template = env.get_template("base.html.jinja")

# 2) Данные для подстановки
context = {
    "name": "Иван Петров",
    "strengths": [
        "Глубокая экспертиза в Python и FastAPI",
        "Быстро закрывает инциденты S2–S3",
        "Качественная коммуникация с заказчиками"
    ],
    "weaknesses": [
        "Недостаточное покрытие автотестами",
        "Задержки с документированием сложных изменений"
    ],
    "suggestions": [
        "Планировать время на документацию в каждом спринте",
        "Добавить интеграционные тесты на критичные эндпоинты",
        "Раз в неделю проводить парные code review"
    ]
}

# 3) Рендерим HTML
html = template.render(**context)

# 4) Сохраняем HTML (опционально — удобно для отладки)
os.makedirs("out", exist_ok=True)
html_path = os.path.join("out", "review_ivan-petrov.html")
with open(html_path, "w", encoding="utf-8") as f:
    f.write(html)



In [ ]:
# 5) Конвертация HTML -> PDF (WeasyPrint)
from weasyprint import HTML

pdf_path = os.path.join("out", "review_ivan-petrov.pdf")
# base_url важен, если в шаблоне есть относительные пути к ресурсам (картинки, шрифты, CSS)
HTML(string=html, base_url=os.getcwd()).write_pdf(pdf_path)

print(f"Готово: {pdf_path}")

In [ ]:
# render_report_weasyprint.py
import re
import tempfile
from pathlib import Path

from jinja2 import Environment, FileSystemLoader, select_autoescape
from weasyprint import HTML

def slugify(s: str) -> str:
    return re.sub(r'[^a-zA-Z0-9_-]+', '-', s.strip()).strip('-').lower()

# 1) Jinja окружение и шаблон
TEMPLATES_DIR = Path("/mnt/d/vscode_projects/praxis-core/jinja_templates")
env = Environment(
    loader=FileSystemLoader(str(TEMPLATES_DIR)),
    autoescape=select_autoescape(["html", "xml"])
)
template = env.get_template("base.html.jinja")

# 2) Данные
context = {
    "name": "Иван Петров",
    "strengths": [
        "Глубокая экспертиза в Python и FastAPI",
        "Быстро закрывает инциденты S2–S3",
        "Качественная коммуникация с заказчиками"
    ],
    "weaknesses": [
        "Недостаточное покрытие автотестами",
        "Задержки с документированием сложных изменений"
    ],
    "suggestions": [
        "Планировать время на документацию в каждом спринте",
        "Добавить интеграционные тесты на критичные эндпоинты",
        "Раз в неделю проводить парные code review"
    ]
}

# 3) Рендерим HTML
html = template.render(**context)

# 4) Сохраняем HTML во временную директорию
tmp_root = Path(tempfile.gettempdir()) / "employee_reviews_html"
tmp_root.mkdir(parents=True, exist_ok=True)
slug = slugify(context["name"]) or "employee"
html_path = tmp_root / f"review-{slug}.html"
html_path.write_text(html, encoding="utf-8")

# 5) Генерируем PDF
out_dir = Path("out"); out_dir.mkdir(parents=True, exist_ok=True)
pdf_path = out_dir / f"review-{slug}.pdf"

# Читаем из файла HTML (чтобы явно использовать сохранённый промежуточный файл)
HTML(filename=str(html_path)).write_pdf(str(pdf_path))

print(f"HTML сохранён во временную папку: {html_path}")
print(f"PDF сохранён: {pdf_path}")

In [ ]:
from typing import List, Tuple, Optional
import numpy as np
import matplotlib.pyplot as plt
from textwrap import wrap
import math

def plot_360_radar(
    pairs: List[Tuple[str, float]],
    title: Optional[str] = "Оценка 360",
    value_range: Optional[Tuple[float, float]] = None,
    show_values: bool = True,
    save_to: Optional[str] = None,
    dpi: int = 200,
    wrap_width: int = 18,
    top_area: float = 0.86,
    title_y: float = 0.985,
    label_radius: float = 1.12
):
    if not pairs or len(pairs) < 3:
        raise ValueError("Нужно минимум 3 дисциплины.")
    labels, values = zip(*pairs)
    values = np.asarray(values, dtype=float)

    fig_bg = "#FBFCFE"
    ax_bg = "#F7F9FC"
    grid_c = "#D6DEE6"
    txt_c = "#2D3A45"
    edge_c = "#4C84B5"
    fill_c = "#4C84B5"

    def nice_max(x: float) -> float:
        if x <= 5.5:
            return 5.0
        if x <= 10.5:
            return 10.0
        if x <= 100.5:
            return 100.0
        mag = 10 ** math.floor(math.log10(x))
        return math.ceil(x / mag) * mag

    if value_range is None:
        vmin, vmax = 0.0, nice_max(np.nanmax(values))
    else:
        vmin, vmax = value_range
        if vmax <= vmin:
            raise ValueError("value_range: max должен быть больше min.")
    rng = max(vmax - vmin, 1e-12)
    r = np.clip((values - vmin) / rng, 0, 1)

    N = len(labels)
    angles = np.linspace(0, 2 * np.pi, N, endpoint=False)
    angles_closed = np.r_[angles, angles[0]]
    r_closed = np.r_[r, r[0]]

    fig = plt.figure(figsize=(7.6, 7.6), dpi=dpi)
    fig.patch.set_facecolor(fig_bg)
    ax = fig.add_subplot(111, projection="polar")
    ax.set_facecolor(ax_bg)

    fig.subplots_adjust(left=0.12, right=0.88, bottom=0.12, top=top_area)

    if title:
        fig.suptitle(title, y=title_y, color=txt_c, fontsize=16, fontweight="semibold")

    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    ax.set_ylim(0, 1.0)

    ax.spines["polar"].set_visible(False)
    ax.yaxis.grid(True, color=grid_c, lw=0.8, alpha=0.85)
    ax.xaxis.grid(False)

    ring_levels = [0.25, 0.5, 0.75, 1.0]
    ring_labels = [f"{(vmin + lvl * rng):.0f}" if rng > 5 else f"{(vmin + lvl * rng):g}" for lvl in ring_levels]
    ax.set_yticks(ring_levels)
    ax.set_yticklabels(ring_labels, color="#8EA0B5", fontsize=9)

    ax.set_xticks([])
    def two_lines(s: str, width: int) -> str:
        parts = wrap(s, width=width, break_long_words=False)
        if len(parts) <= 1:
            return s
        return parts[0] + "\n" + " ".join(parts[1:])
    labels2 = [two_lines(lbl, wrap_width) for lbl in labels]
    for ang, text in zip(angles, labels2):
        ax.text(ang, label_radius, text, color=txt_c, fontsize=11, ha="center", va="center", clip_on=False)

    ax.plot(angles_closed, r_closed, color=edge_c, lw=7, alpha=0.08, solid_capstyle="round")
    ax.plot(angles_closed, r_closed, color=edge_c, lw=2.6, solid_capstyle="round")
    ax.fill(angles_closed, r_closed, color=fill_c, alpha=0.16)

    ax.scatter(angles, r, s=38, color=fig_bg, edgecolor=edge_c, linewidth=2, zorder=5)

    if show_values:
        for ang, rr, val in zip(angles, r, values):
            txt = f"{val:.0f}" if rng > 5 else f"{val:g}"
            ax.annotate(txt, xy=(ang, rr), xytext=(0, 0), textcoords="offset points", ha="center", va="center", fontsize=10, color=txt_c)

    fig.canvas.draw()
    if save_to:
        fig.savefig(save_to, dpi=dpi, facecolor=fig.get_facecolor(), transparent=False)
    else:
        plt.show()
    plt.close(fig)


if __name__ == "__main__":
    data = [
        ("Стратегическое мышление", 82),
        ("Коммуникация", 74),
        ("Командная работа", 91),
        ("Клиентоориентированность", 86),
        ("Лидерство", 79),
        ("Ответственность", 88),
    ]
    plot_360_radar(
        data,
        title="Оценка 360 — Иван Иванов",
        # label_radius=1.13,
        save_to="360_radar.png"
    )

In [ ]:
from typing import List, Tuple, Optional
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from textwrap import wrap
import math

def plot_360_radar_dual(
    pairs_self: List[Tuple[str, float]],
    pairs_mgr: List[Tuple[str, float]],
    title: Optional[str] = "Оценка 360",
    value_range: Optional[Tuple[float, float]] = None,
    save_to: Optional[str] = None,
    dpi: int = 200,
    wrap_width: int = 18,
    top_area: float = 0.86,
    bottom_area: float = 0.14,
    title_y: float = 0.985,
    label_radius: float = 1.12,
    show_values: bool = False
):
    if not pairs_self or not pairs_mgr or len(pairs_self) < 3 or len(pairs_mgr) < 3:
        raise ValueError("Нужно минимум 3 дисциплины в каждом списке.")
    labels_s, values_s = zip(*pairs_self)
    labels_m, values_m = zip(*pairs_mgr)
    if set(labels_s) != set(labels_m):
        raise ValueError("Наборы дисциплин в самооценке и оценке руководства должны совпадать.")
    order = [lbl for lbl, _ in pairs_mgr]
    d_self = dict(pairs_self)
    d_mgr = dict(pairs_mgr)
    labels = order
    values_self = np.asarray([d_self[lbl] for lbl in labels], dtype=float)
    values_mgr = np.asarray([d_mgr[lbl] for lbl in labels], dtype=float)

    fig_bg = "#FBFCFE"
    ax_bg = "#F7F9FC"
    grid_c = "#D6DEE6"
    txt_c = "#2D3A45"
    mgr_c = "#4C84B5"
    self_c = "#5AA6B0"

    def nice_max(x: float) -> float:
        if x <= 5.5:
            return 5.0
        if x <= 10.5:
            return 10.0
        if x <= 100.5:
            return 100.0
        mag = 10 ** math.floor(math.log10(x))
        return math.ceil(x / mag) * mag

    if value_range is None:
        vmax = nice_max(float(np.nanmax(np.r_[values_self, values_mgr])))
        vmin = 0.0
    else:
        vmin, vmax = value_range
        if vmax <= vmin:
            raise ValueError("value_range: max должен быть больше min.")
    rng = max(vmax - vmin, 1e-12)

    r_self = np.clip((values_self - vmin) / rng, 0, 1)
    r_mgr = np.clip((values_mgr - vmin) / rng, 0, 1)

    N = len(labels)
    angles = np.linspace(0, 2 * np.pi, N, endpoint=False)
    angles_closed = np.r_[angles, angles[0]]
    r_self_closed = np.r_[r_self, r_self[0]]
    r_mgr_closed = np.r_[r_mgr, r_mgr[0]]

    fig = plt.figure(figsize=(7.8, 7.8), dpi=dpi)
    fig.patch.set_facecolor(fig_bg)
    ax = fig.add_subplot(111, projection="polar")
    ax.set_facecolor(ax_bg)

    fig.subplots_adjust(left=0.10, right=0.90, bottom=bottom_area, top=top_area)
    if title:
        fig.suptitle(title, y=title_y, color=txt_c, fontsize=16, fontweight="semibold")

    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    ax.set_ylim(0, 1.0)

    ax.spines["polar"].set_visible(False)
    ax.yaxis.grid(True, color=grid_c, lw=0.8, alpha=0.85)
    ax.xaxis.grid(False)

    ring_levels = [0.25, 0.5, 0.75, 1.0]
    ring_labels = [f"{(vmin + lvl * rng):.0f}" if rng > 5 else f"{(vmin + lvl * rng):g}" for lvl in ring_levels]
    ax.set_yticks(ring_levels)
    ax.set_yticklabels(ring_labels, color="#8EA0B5", fontsize=9)

    def two_lines(s: str, width: int) -> str:
        parts = wrap(s, width=width, break_long_words=False)
        if len(parts) <= 1:
            return s
        return parts[0] + "\n" + " ".join(parts[1:])
    labels2 = [two_lines(lbl, wrap_width) for lbl in labels]
    ax.set_xticks([])
    for ang, text in zip(angles, labels2):
        ax.text(ang, label_radius, text, color=txt_c, fontsize=11, ha="center", va="center", clip_on=False)

    ax.plot(angles_closed, r_mgr_closed, color=mgr_c, lw=6, alpha=0.06, solid_capstyle="round")
    ax.plot(angles_closed, r_mgr_closed, color=mgr_c, lw=2.4, alpha=0.85, solid_capstyle="round")
    ax.fill(angles_closed, r_mgr_closed, color=mgr_c, alpha=0.12)

    ax.plot(angles_closed, r_self_closed, color=self_c, lw=6, alpha=0.06, solid_capstyle="round")
    ax.plot(angles_closed, r_self_closed, color=self_c, lw=2.4, alpha=0.85, solid_capstyle="round")
    ax.fill(angles_closed, r_self_closed, color=self_c, alpha=0.12)

    ax.scatter(angles, r_mgr, s=32, color=fig_bg, edgecolor=mgr_c, linewidth=2, zorder=5, alpha=0.9)
    ax.scatter(angles, r_self, s=32, color=fig_bg, edgecolor=self_c, linewidth=2, zorder=5, alpha=0.9)

    if show_values:
        for ang, rr, val in zip(angles, r_mgr, values_mgr):
            ax.annotate(f"{val:.0f}", xy=(ang, rr), xytext=(0, 0), textcoords="offset points", ha="center", va="center", fontsize=10, color=txt_c)
        for ang, rr, val in zip(angles, r_self, values_self):
            ax.annotate(f"{val:.0f}", xy=(ang, rr), xytext=(0, -12), textcoords="offset points", ha="center", va="center", fontsize=9, color=txt_c)

    handles = [
        Line2D([0], [0], color=mgr_c, lw=2.6, marker="o", markersize=5, markerfacecolor=fig_bg, markeredgecolor=mgr_c, alpha=0.9),
        Line2D([0], [0], color=self_c, lw=2.6, marker="o", markersize=5, markerfacecolor=fig_bg, markeredgecolor=self_c, alpha=0.9),
    ]
    fig.legend(handles, ["Руководство", "Самооценка"], loc="lower center", bbox_to_anchor=(0.5, 0.03), ncol=2, frameon=False)

    fig.canvas.draw()
    if save_to:
        fig.savefig(save_to, dpi=dpi, facecolor=fig.get_facecolor(), transparent=False)
    else:
        plt.show()
    plt.close(fig)


if __name__ == "__main__":
    self_data = [
        ("Стратегическое мышление", 78),
        ("Коммуникация", 80),
        ("Командная работа", 88),
        ("Клиентоориентированность", 83),
        ("Лидерство", 74),
        ("Ответственность", 90),
    ]
    mgr_data = [
        ("Стратегическое мышление", 82),
        ("Коммуникация", 74),
        ("Командная работа", 91),
        ("Клиентоориентированность", 86),
        ("Лидерство", 79),
        ("Ответственность", 88),
    ]
    plot_360_radar_dual(
        self_data,
        mgr_data,
        title="Оценка 360 — Иван Иванов",
        save_to="360_radar_dual.png"
    )